In [63]:
def spatial_decomposition(X, unit_cell, *args):
    '''
    % decomposite the spatial domain into cells D with vertices V,
    %
    % Output
    %  V - list of vertices
    %  F - list of faces
    %  I_FD - incidence matrix between faces to cells

    % compute voronoi decomposition
    % V - list of vertices of the Voronoi cells
    % D   - cell array of Vornoi cells with centers X_D ordered accordingly
    '''
    # Imports
    import getopt
    import numpy as np
    from numpy import cumsum, zeros, unique, sort
    from spatialDecompFunctions import generateUnitCells
    from scipy.spatial import Voronoi
    from scipy.sparse import csr_matrix
    
    if unit_cell.all() == None:
        unit_cell = calcUnitCell(X)
    
    if args[0] == 'unit_cell':
    
        # compute the vertices
        [V, faces] = generateUnitCells(X, unit_cell, args[:])
        
        D = np.empty(len(X),dtype=object)

        for k in range(X.shape[0]):
            D[k] = faces[k, :]
    
    else:    
        var_arg_in = args[0]
        dummyCoordinates = calcBoundary(X, unit_cell, var_arg_in)

        [V,D] = Voronoi(np.array([[X, dummyCoordinates]]).T, 
                        qhull_options = {'Q5','Q6','Qs'}
            ) #,'QbB'

        D = D[1:X.shape[0]]

    # now we need some adjacencies and incidences
    iv = [D[:]]            # nodes incident to cells D
    id = zeros(len(iv))   # number the cells
        
    # Some MATLAB stuff goin on here... : p = np.array([[0, cumsum(cellfun('prodofsize',D))]]).T
    D_prod = 1
    for elem in D.shape:
        D_prod *= elem
    p = np.array( [[0, cumsum(D_prod)]] ).T
    for k in range(len(D)):
        id[p(k)+1 : p(k+1)] = k
        
    # next vertex
    ind_x = list( range(2,len(iv)+1) )
    ind_x[p[2:]] = p[1:-2] + 1
    iv_n = iv(ind_x)

    # edges list
    F = [iv[:], iv_n[:]]

    # should be unique (i.e one edge is incident to two cells D)
    [F, _, ie] = unique(sort(F,axis=1), axis=0)

    # faces incident to cells, F x D
    #I_FD = sparse(ie,id,1);
    I_FD = csr_matrix( (ie, id, 1) ) # could also use csc_matrix() if it improves
                                 # performance !
    

    return V, F, I_FD


# numpy.loadtxt
def calcBoundary(X, unit_cell, var_arg_in='hull'):
    '''
    dummy coordinates so that the voronoi-cells of X are finite

    Inputs:
    --------------
    X : n x 2 numpy array of [x,y] vertex coordinates

    unit_cell : n x 2 numpy array of [x,y] coordinates of "pixel" boundary (e.g. hexagon or square)

    var_arg_in : ???
    ???


    Outputs:
    --------------
    ??? : ???
    ???

    Dependencies:
    --------------
    import getopt
    import numpy as np
    from scipy.spatial import ConvexHull
    from numpy import arctan2, diff, cumsum, matrix, squeeze, unique
    from statistics import mean
    from math import sqrt, floor, copysign
    '''
    # Imports
    import getopt
    import numpy as np
    from scipy.spatial import ConvexHull
    from numpy import arctan2, diff, cumsum, matrix, squeeze, unique, linspace
    from statistics import mean
    from math import sqrt, floor, copysign
    import orix
    
    from spatialDecompFunctions import householderMatrix, translationMatrix, erase_linearly_dependent_points

    dummy_coordinates = []

    boundary = str(var_arg_in)

    print(boundary)
    if boundary.isalpha():

        if (boundary.lower() == 'hull' or
            boundary.lower() == 'convexhull'): 

            k = ConvexHull(X)
            
            # erase all linearly dependent points
            angle = arctan2( 
                x[k[0:-2]]-x[k[1:]],
                y[k[0:-2]]-y[k[1:]] )
            
            k = erase_linearly_dependent_points(k.simplices)
            bounding_X = X[k, :]
            
        elif boundary.lower() == 'cube':
            # set up a rectangular box
            envelope_X = [np.amin(X), np.amax(X)]
            bounding_X = [
                envelope_X[1], envelope_X[3],
                envelope_X[2], envelope_X[3],
                envelope_X[2], envelope_X[4],
                envelope_X[1], envelope_X[4],
                envelope_X[1], envelope_X[3],
                ]

        else:
            raise ValueError('Unknown boundary type. Available options are \
            ''hull'', ''convexhull'' and ''cube''.')

    elif isinstance(boundary, float):
        bounding_X = boundary
    
    radius = mean( sum( (unit_cell**2) ** 0.5 ) )
    edge_length = sum( diff( (bounding_X**2) ** 0.5 ) ) 

    # fill each line segment with nodes every 20 points (in average)
    nto = floor( (edge_length>0)*4 )
    nto = floor( edge_length*(2*radius) )

    cs = cumsum([1, 1 + nto])
    bounding_X[cs, :] = bounding_X

    # interpolation
    for k in range(len(nto)):
        for dim in range(2):
            bounding_X[cs[k]:cs[k+1], dim] = linspace(
                bounding_X(cs[k],dim),
                bounding_X(cs[k+1], dim),
                nto(k)+2
            )

    # homogeneous coordinates
    X[:, 3] = 1
    bounding_X[:, 3] = 1

    # householder matrix
    H = householderMatrix(v)
    # translation matrix
    T = translationMatrix(s)

    # direction of the edge
    edge_direction = diff(bounding_X)
    edge_angle = arctan2( edge_direction[:, 2], edge_direction[:,1] )
    edge_length = sqrt( sum(edge_direction**2, axis = 1) )

    # shift the starting vertex
    b_X = squeeze( float( orix.quaternion.Rotation.from_axes_angles(np.array([0, 0, 1]), edge_angle), \
        np.array([[0, radius, 1]]).T ) )
    offset_X = b_X - bounding_X[1:-2, :]

    for k in range(bounding_X.shape[0]-1):
        
        # mirror the point set X on each edge
        p_X = X * -1*( T[offset_X[k, :]] * H[edge_direction[k, :]] * T[offset_X[k, :]] ).T
        
        # distance between original and mirrored point
        dist = sqrt( sum( (X[:,1:2]-p_X[:,1:2])**2, axis = 1 ) )

        intend_X = 2 * radius * copysign(1, edge_direction[k,1:2])
        
        # now try to delete unnecessary points
        m = 2

        condition1 = True
        while condition1:
        
            tmp_X = p_X[dist < m*radius,1:2]

            right = tmp_x[np.tile(tmp_X) - ( bounding_X[k, 0:2]   - intend_X ) * edge_direction[k, 1:2].T < 0]
            left  = tmp_x[np.tile(tmp_X) - ( bounding_X[k+1, 0:2] + intend_X ) * edge_direction[k, 1:2].T < 0]
            
            tmp_X = tmp_X[ not(right or left), :]
            
            if edge_length(k) / tmp_X.shape[0] < radius/3:
                break
            elif m < 2**7:
                m = m*2
            else:
                m = m+10
        
        dummy_coordinates = np.array([[dummy_coordinates, tmp_X]]).T
        
    _, dummy_coordinates, _, _ = unique(dummyCoordinates, axis = 0)

def erase_linearly_dependent_points(points):
    '''
    subfunction to remove linearly dependent points.

    Inputs:
    --------------
    k : ???
        ???

    Outputs:
    --------------
    ??? : ???
        ???

    Dependencies:
    --------------
    from scipy.spatial import ConvexHull
    '''
    from orix.utilities import regularPoly
    import numpy as np
    from scipy.spatial import ConvexHull
    k = ConvexHull(points)

    # erase all linear dependent points
    angle = np.arctan2( 
        x[k.vertices[0:-1]]-x[k.vertices[1:]],
        y[k.vertices[0:-1]]-y[k.vertices[1:]]
    )
    test = np.abs(np.diff(angle))>np.spacing(1.0)
    k2 = k.vertices[np.concatenate([[True], test, [True]])]
    boundingX = [x[k2], y[k2]]

    cellRot=0
    
    unitCell = regularPoly(4,xstp,cellRot)

    print(unitCell)
    radius = np.mean( np.sqrt(np.sum(unitCell**2,2)) )
    edgeLength = np.sqrt( np.sum( np.diff(boundingX)**2, axis = 2) )

    return k


In [ ]:
X = np.loadtxt('spatialDecomposition_input_x.csv', delimiter=',', dtype=float)
unit_cell = np.loadtxt('spatialDecomposition_input_unitCell.csv', delimiter=',', dtype=float)
spatial_decomposition(X, unit_cell, 'hull')

In [82]:
import getopt
import numpy as np
from scipy.spatial import ConvexHull
from numpy import arctan2, diff, cumsum, matrix, squeeze, unique, linspace
from statistics import mean
from math import sqrt, floor, copysign
import orix

from spatialDecompFunctions import householderMatrix, translationMatrix, erase_linearly_dependent_points

x = X[:,0]
y = X[:,1]

k = ConvexHull(X)
k = erase_linearly_dependent_points(k.simplices)

NameError: name 'x' is not defined

In [81]:
k.simplices

array([[106759, 106421],
       [106759,    339],
       [106420,      0],
       [106420, 106421],
       [   338,      0],
       [   338,    339]], dtype=int32)